In [1]:
!pip install --upgrade 'chromadb==0.3.26' 'pydantic==1.10.0' sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.0 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 42.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.5/977.5 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 66.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 68.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 75.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 75.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 78.3 MB/s eta 0:00:00:00

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0


## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [2]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
    "apikey": "WNTEfrjfUygDMK5_8eW-TMGYGUgNgxw2aor4bWsb0Wit"
    }


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [3]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [4]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 500,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [5]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [6]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


## Defining the vector index
Initialize the vector index to query when chatting with the model.

In [7]:
from ibm_watsonx_ai.client import APIClient

wml_credentials = get_credentials()
client = APIClient(credentials=wml_credentials, project_id=project_id, space_id=space_id)

vector_index_id = "60271900-118a-4595-9d65-1cb60d22d413"
vector_index_details = client.data_assets.get_details(vector_index_id)
vector_index_properties = vector_index_details["entity"]["vector_index"]

In [8]:
vector_index_details

{'metadata': {'project_id': '5637c821-378b-4fc9-b2b7-c96b62f8be4e',
  'sandbox_id': '5637c821-378b-4fc9-b2b7-c96b62f8be4e',
  'usage': {'last_updated_at': '2024-09-30T00:13:54Z',
   'last_updater_id': 'IBMid-695000JECA',
   'last_update_time': 1727655234663,
   'last_accessed_at': '2024-09-30T00:13:54Z',
   'last_access_time': 1727655234663,
   'last_accessor_id': 'IBMid-695000JECA',
   'access_count': 0},
  'rov': {'mode': 0,
   'collaborator_ids': {},
   'member_roles': {'IBMid-695000JECA': {'user_iam_id': 'IBMid-695000JECA',
     'roles': ['OWNER']}}},
  'name': 'المبتدأ',
  'description': '',
  'asset_type': 'vector_index',
  'rating': 0.0,
  'total_ratings': 0,
  'catalog_id': '2df865ef-e5a5-4360-816f-2c36061723d3',
  'created': 1727655134852,
  'created_at': '2024-09-30T00:12:14Z',
  'owner_id': 'IBMid-695000JECA',
  'size': 25832,
  'version': 2.0,
  'asset_state': 'available',
  'asset_attributes': ['vector_index'],
  'asset_id': '60271900-118a-4595-9d65-1cb60d22d413',
  'asset

In [9]:
from ibm_watsonx_ai.foundation_models.embeddings.sentence_transformer_embeddings import SentenceTransformerEmbeddings

emb = SentenceTransformerEmbeddings('sentence-transformers/all-MiniLM-L6-v2')

import subprocess
import gzip
import json
import chromadb
import random
import string

def hydrate_chromadb():
    data = client.data_assets.get_content(vector_index_id)
    content = gzip.decompress(data)
    stringified_vectors = str(content, "utf-8")
    vectors = json.loads(stringified_vectors)

    chroma_client = chromadb.Client()

    # make sure collection is empty if it already existed
    collection_name = "my_collection"
    try:
        collection = chroma_client.delete_collection(name=collection_name)
    except:
        print("Collection didn't exist - nothing to do.")
    collection = chroma_client.create_collection(name=collection_name)

    vector_embeddings = []
    vector_documents = []
    vector_metadatas = []
    vector_ids = []

    for vector in vectors:
        vector_embeddings.append(vector["embedding"])
        vector_documents.append(vector["content"])
        metadata = vector["metadata"]
        lines = metadata["loc"]["lines"]
        clean_metadata = {}
        clean_metadata["asset_id"] = metadata["asset_id"]
        clean_metadata["asset_name"] = metadata["asset_name"]
        clean_metadata["url"] = metadata["url"]
        clean_metadata["from"] = lines["from"]
        clean_metadata["to"] = lines["to"]
        vector_metadatas.append(clean_metadata)
        asset_id = vector["metadata"]["asset_id"]
        random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
        id = "{}:{}-{}-{}".format(asset_id, lines["from"], lines["to"], random_string)
        vector_ids.append(id)

    collection.add(
        embeddings=vector_embeddings,
        documents=vector_documents,
        metadatas=vector_metadatas,
        ids=vector_ids
    )
    return collection

chroma_collection = hydrate_chromadb()

def proximity_search( question ):
    query_vectors = emb.embed_query(question)
    query_result = chroma_collection.query(
        query_embeddings=query_vectors,
        n_results=vector_index_properties["settings"]["top_k"],
        include=["documents", "metadatas", "distances"]
    )

    documents = list(reversed(query_result["documents"][0]))

    return "\n".join(documents)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Collection didn't exist - nothing to do.


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [44]:
old_prompt_input = """
__grounding__

<<SYS>>
لنلعب لعبة تعليمية تفاعلية للغة العربية للأطفال، تركز على بناء الجمل الاسمية البسيطة. اللعبة تتكون من سلسلة من الأسئلة التي تتطلب من اللاعب إكمال جملة تبدأ بأحد أشكال المبتدأ.

ابدأ اللعبة أنت بأن تسأل المستخدم عن اسمه والمستوى المطلوب
ألق التحية على المستخدم باستخدام اسمه
تأكد أن المتعلم اختار واحد من المستويات المتاحة فقط وهي: (الأول).
إذا اختار المتعلم المستوى الأول فاتبع التعليمات التالية:
سنساعد المتعلم على فهم قاعدة المبتدأ المعرف بأل التعريف.
 اكتب جملة ناقصة يطلب من اللاعب إكمالها بكلمة مناسبة تأتي في بداية الجملة.
 اختر السؤال مراعياً كون الإجابات المحتملة منطقية ومتعلقة بموضوع الجملة.
الجواب:
إذا أجاب المتعلم جواباً صحيحاً فشجعه بعبارة تحفيزية صغية ثم انتقل للسؤال التالي
 إذا أجاب المتعلم جواب خاطئ فقم بمساعدته على تخمين الجواب الصحيح عن طريق طرح سؤال يتعلق بالجواب الصحيح 
إذا كتب المتعلم "تابع" انتقل للسؤال التالي
إذا كتب المتعلم "لا أعرف" فأعطه الجواب الصحيح ثم انتقل للسؤال التالي


تنسيق الخرج:
قدم لي الخرج بتنسيق JSON، حيث يكون لكل سؤال كائن يحتوي على الحقول التالية:
السؤال: وهو النص الكامل للسؤال مع الفراغ.
الإجابة_الصحيحة: وهي الكلمة الصحيحة التي تكمل الجملة.
الأسئلة_المساعدة: وهي قائمة من ثلاثة أسئلة هدفها مساعدة المتعلم على تخمين الإجابة الصحيحة.
الشرح: وهو شرح بسيط للقاعدة النحوية.

لا تعرض للمتعلم الأمثلة المذكورة أعلاه

<</SYS>>

"""


In [ ]:
prompt_input = """
__grounding__

<<SYS>>
لنلعب لعبة تعليمية تفاعلية للغة العربية للأطفال، تركز على بناء الجمل الاسمية البسيطة. اللعبة تتكون من سلسلة من الأسئلة التي تتطلب من اللاعب إكمال جملة تبدأ بأحد أشكال المبتدأ.

اللعبة:
تبدأ اللعبة بأن يخبرك اللاعب باسمه.
أجب اللاعب بأن ترحب به ب5 كلمات فقط، واطلب منه تحديد أي مستوى يريد أن يلعب.
يكتب اللاعب "الأول" ، بعدها أخبره أن يكتب "أبدأ" لتبدأ اللعبة.
عندما يكتب اللاعب "أبدأ" قم بتوليد وطباعة قائمة أحرف الجر العشوائية التي تدخل في المستوى المختار بالطول المحدد للقائمة حسب المستوى المحدد
تأكد أن المتعلم اختار واحد من المستويات المتاحة فقط وهي: (الأول

إذا اختار المتعلم المستوى الأول فاتبع التعليمات التالية:
سنساعد المتعلم على فهم قاعدة المبتدأ المعرف بأل التعريف.
 اكتب جملة ناقصة يطلب من اللاعب إكمالها بكلمة مناسبة تأتي في بداية الجملة.
 اختر السؤال مراعياً كون الإجابات المحتملة منطقية ومتعلقة بموضوع الجملة.
إذا أجاب المتعلم جواباً صحيحاً فشجعه بعبارة تحفيزية صغيرة ثم انتقل للسؤال التالي
 إذا أجاب المتعلم جواب خاطئ فقم بمساعدته على تخمين الجواب الصحيح عن طريق طرح سؤال يتعلق بالجواب الصحيح 
إذا كتب المتعلم "تابع" انتقل للسؤال التالي
إذا كتب المتعلم "لا أعرف" فأعطه الجواب الصحيح ثم انتقل للسؤال التالي


تنسيق الخرج:
قدم لي الخرج بتنسيق JSON، حيث يكون لكل سؤال كائن يحتوي على الحقول التالية:
السؤال: وهو النص الكامل للسؤال مع الفراغ.
الإجابة_الصحيحة: وهي الكلمة الصحيحة التي تكمل الجملة.
الأسئلة_المساعدة: وهي قائمة من ثلاثة أسئلة هدفها مساعدة المتعلم على تخمين الإجابة الصحيحة.
الشرح: وهو شرح بسيط للقاعدة النحوية.

لا تعرض للمتعلم الأمثلة المذكورة أعلاه

<</SYS>>

"""


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [47]:
user_input = input("اسمك: ")
while user_input != "توقف":
    grounding = proximity_search(user_input)
    formattedQuestion = f"""<s> [INST] {user_input} [/INST]"""
    prompt = f"""{prompt_input}{formattedQuestion}"""
    generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
    print(f"AI: {generated_response}")
    user_input = input("إجابتك: ")

المتعلم: الشمس
المساعد:  مرحبًا يا [اسم_المستخدم]! سأقوم بتصميم لعبة تعليمية تفاعلية باللغة العربية للأطفال تركز على بناء الجمل الاسمية البسيطة. سنبدأ بالمستوى الأول الذي يركز على المبتدأ المعرف بأل التعريف. إليك سؤالاً بسيطاً:

سؤال: في صباحٍ جميلٍ، تألقت الشمسُ في السماء.

الإجابة_الصحيحة: الشمس

الأسئلة_المساعدة:
1. ما هو الاسم الذي يأتي بعد "في صباحٍ جميلٍ"؟
2. ما هو العنصر الذي يظهر في السماء عادة؟
3. ما هو الاسم الذي يبدأ بـ"أل" ويعني النجم الكبير الذي تضيء السماء؟

الشرح: المبتدأ هو الاسم الذي يبدأ به الجملة الاسمية ويأتي بعده الخبر ليكمل معنى الجملة. في هذا المثال، "الشمس" هو المبتدأ المعرف بأل التعريف.

يرجى إعلامي إذا كنت ترغب في متابعة اللعب أو إذا كنت بحاجة إلى مساعدة إضافية. 


In [41]:
question = input("المتعلم: ")
grounding = proximity_search(question)
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
print(f"المساعد: {generated_response}")


المتعلم: مروة
المساعد:  أهلاً بك يا مروة! سأصمم لك لعبة تعليمية تفاعلية تركز على بناء الجمل الاسمية البسيطة باللغة العربية. دعنا نبدأ المستوى الأول الذي يركز على المبتدأ المعرف بأل التعريف.

فيما يلي مثال على تنسيق الأسئلة التي ستُطرح عليك:

```json
{
  "السؤال": "الشمسُ ساطعة.",
  "الإجابة_الصحيحة": "الشمس",
  "الأسئلة_المساعدة": [
    "ما هو الاسم الذي يأتي بعد 'أل' التعريف في هذه الجملة؟",
    "ما هو المبتدأ الذي يأتي قبل الخبر 'ساطعة' في هذه الجملة؟",
    "كيف يمكن إكمال هذه الجملة بشكل صحيح؟"
  ],
  "الشرح": "في هذه الجملة، يأتي المبتدأ معرفاً بأل التعريف وهو 'الشمس'. 'أل' التعريف تستخدم لتعريف الاسم وتمييزه عن غيره من الأسماء."
}
```

عند الإجابة على السؤال، سأقدم لك التوجيه والتشجيع. إذا كنتِ مستعدة، لنبدأ اللعبة! 


In [35]:
question = input("المتعلم: ")
grounding = proximity_search(question)
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
print(f"المساعد: {generated_response}")


المتعلم: هي
المساعد:  شكراً لتجاوبك. دعنا نستمر في اللعبة التعليمية لفهم قواعد المبتدأ والخبر باللغة العربية. سأقدم لك سؤالاً آخر يتعلق بالمبتدأ والخبر والمستوى الأول:

السؤال: ... طالب مجد.

الإجابة_الصحيحة: أنا (أو أنت إذا كنت تتحدث مع المتعلم)

الأسئلة_المساعدة:
1. إذا كنت تتحدث عن نفسك، كيف تصفها؟ استخدم الضمير المناسب.
2. تخيل أنك تتحدث عن صديقك، كيف تصف جهوده؟ استخدم الضمير المناسب.
3. ما هو الضمير الذي يشير إلى المتكلم أو المخاطب المفرد؟

الشرح: المبتدأ هو اسم مرفوع تبدأ به الجملة الاسمية ويأتي على شكل ضمير. في هذه الحالة، يجب استخدام ضمير المتكلم (أنا) أو المخاطب (أنت) ليكون المبتدأ.

يرجى متابعة الإجابة على الأسئلة ومساعدتي في تقديم المزيد من الأسئلة والتوضيحات حسب تقدمك في اللعبة. 


In [24]:
question = input("المتعلم: ")
grounding = proximity_search(question)
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
print(f"المساعد: {generated_response}")


المتعلم: اسألني
المساعد:  مرحباّ، أنا مساعد ذكاء صناعي لمساعدتك في تعلم اللغة العربية وتحسين مهاراتك. سأبدأ معك بسؤال بسيط حول المبتدأ الذي تعلمناه سابقاً:

سؤال: ما هو المبتدأ وكيف يمكن تعريفه باللغة العربية؟

يرجى الإجابة على السؤال وسأكون سعيداً لمساعدتك في فهم الإجابة الصحيحة وتطوير معرفتك حول المبتدأ وقواعده. 


In [17]:
question = input("المتعلم: ")
grounding = proximity_search(question)
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
print(f"المساعد: {generated_response}")


المتعلم: هل إجابتي صحيحة؟
المساعد:  كمساعد ذكاء اصطناعي، أنا هنا لمساعدتك وتوجيهك. لكن لم تقدم لي إجابة محددة لتقييمها. إذا كنت ترغب في الحصول على مساعدة بخصوص لعبة تعليمية تفاعلية للغة العربية للأطفال أو موضوع معين يتعلق بالقواعد النحوية، يرجى إعلامي وسأكون سعيداً لمساعدتك. 


In [18]:
question = input("المتعلم: ")
grounding = proximity_search(question)
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
print(f"المساعد: {generated_response}")


المتعلم: سألتني عن إتمام جملة ... مفيد وأعطيتك إجابة الكتاب، هل إجابتي صحيحة؟
المساعد:  في سياق تعليم اللغة العربية وبناء الجمل الاسمية البسيطة للأطفال، تعتبر الإجابة "الكتاب" صحيحة إذا كانت الجملة الناقصة تتعلق بموضوع يفيد العلم أو المعرفة. على سبيل المثال:

السؤال: ... مفيد للطلاب.
الإجابة الصحيحة: الكتاب (أو أي اسم معرف آخر مناسب لمعنى الجملة)

في هذه الحالة، "الكتاب" يعتبر مبتدأ معرفة (بالأل) وخبر الجملة الناقصة هو "مفيد للطلاب". لكن يجب مراعاة مستوى التعلم (الأول) والتأكد من أن الجملة تناسب عمر ومستوى المتعلم. 


In [19]:
question = input("المتعلم: ")
grounding = proximity_search(question)
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
print(f"المساعد: {generated_response}")


المتعلم: وماذا وجدت، هل الجملة تناسب المستوى الأول؟
المساعد:  نعم، الجملة التي قدمتها في سؤالك تناسب المستوى الأول من تعليم المبتدأ والخبر للأطفال. الجملة هي:

"العلمُ نافعٌ."

في هذه الجملة:
1. "العلمُ" هو المبتدأ المعرف بأل التعريف.
2. "نافعٌ" هو الخبر المرفوع الذي يخبر عن المبتدأ.

يمكن استخدام هذه الجملة لتعليم الأطفال عن المبتدأ والخبر وكيفية تكوين الجمل الاسمية البسيطة. 

إذا كنت ترغب في إنشاء لعبة تعليمية تفاعلية كما ذكرت سابقاً، يمكنني مساعدتك في تصميم الأسئلة وتوفير المعلومات المناسبة لكل سؤال. 


In [13]:
question = input("المتعلم: ")
grounding = proximity_search(question)
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
print(f"المساعد: {generated_response}")


المتعلم: تابع
المساعد:  شكراً لاختيارك متابعة الدرس. سأقوم الآن بشرح كيفية إنشاء لعبة تعليمية تفاعلية للغة العربية للأطفال تركز على بناء الجمل الاسمية البسيطة. سنستخدم لغة البرمجة بايثون ومكتبة Pygame لإنشاء هذه اللعبة.

1. إعداد البيئة البرمجية:
   - تأكد من تثبيت Python على جهاز الكمبيوتر الخاص بك. يمكنك تحميله من الموقع الرسمي: https://www.python.org/downloads/
   - قم بتثبيت مكتبة Pygame باستخدام الأمر التالي في نافذة الأوامر (Terminal) أو موجه الأوامر (Command Prompt):

     ```
     pip install pygame
     ```

2. إنشاء مشروع جديد:
   - افتح محرر نصوص مثل Visual Studio Code أو Sublime Text.
   - أنشئ ملف جديد باسم `game.py` داخل مجلد المشروع الخاص بك.

3. كتابة الكود الأساسي للعبة:

في ملف `game.py`، سنقوم بكتابة الكود الأساسي للعبة. سنستخدم مكتبة Pygame لإنشاء واجهة المستخدم والتعامل مع إدخال المستخدم. إليك مثال بسيط على كيفية إنشاء نافذة للعب:

```python
import pygame
import sys

# تهيئة مكتبة Pygame
pygame.init()

# تعريف الألوان
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# تع

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  